In [ ]:
#need for kaggle online ipynb
#! pip install imutils

In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
import os
import random
import cv2
import imutils
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelBinarizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras import optimizers
from sklearn.preprocessing import LabelBinarizer
from keras import backend as K
from keras.layers import Dense, Activation, Flatten, Dense,MaxPooling2D, Dropout, SpatialDropout2D, GlobalMaxPooling2D
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
import tensorflow as tf

In [ ]:
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# #make sure to use GPU
# tf.debugging.set_log_device_placement(True)

In [ ]:
# gpus = tf.config.list_physical_devices('GPU')
# gpus

In [ ]:
# for google colab - after you load your kaggle api token you can give it permissions on colab (if using colab)
# !chmod 600 /content/kaggle.json

In [ ]:
# google colab 
# ! kaggle datasets download -d vaibhao/handwritten-characters
# ! kaggle datasets download -d landlord/handwriting-recognition

In [ ]:
# import zipfile #unzipping the dataset (google colab)
# zip_ref = zipfile.ZipFile('handwritten-characters.zip', 'r')
# zip_ref.extractall('/content/datasets/handwritten-characters')
# zip_ref.close()

In [ ]:
# import zipfile #unzipping the dataset (google colab)
# zip_ref = zipfile.ZipFile('handwriting-recognition.zip', 'r')
# zip_ref.extractall('/content/datasets/handwriting-recognition')
# zip_ref.close()

Team used following datasets (images): <br>
In handwritten model (used in final version):
- [Handwritten Characters] https://www.kaggle.com/datasets/vaibhao/handwritten-characters/ (used for training)
- [Handwriting Recognition] https://www.kaggle.com/datasets/landlord/handwriting-recognition (used for testing)

In second model (both were combined into one and split into training and testing sets):
- [Printed Digits] https://www.kaggle.com/datasets/dhruvmomoman/printed-digits
- [Camera-Taken Images of Printed English Alphabet] https://www.kaggle.com/datasets/naderabdalghani/camerataken-images-of-printed-english-alphabet

In [ ]:
# for kaggle (online model):
# dir = "/kaggle/input/handwritten-characters/Train/"

dir = "datasets/handwritten-characters/Train/"
val_dir = "datasets/handwritten-characters/Validation/"


train_data = []
val_data = []
img_size = 32
limit_train_images = 4200
limit_val_images = 1000

non_chars = ["#","$","&","@", "0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]

def get_data(dir, max_images, not_list, data_list):
    for i in os.listdir(dir):
        if i in not_list:
            continue
        count = 0
        sub_directory = os.path.join(dir,i)
        for j in os.listdir(sub_directory):
            count+=1
            if count > max_images: #limiting the number of images per label
                break
            img = cv2.imread(os.path.join(sub_directory,j),0)
            img = cv2.resize(img,(img_size,img_size))
            data_list.append([img,i])


get_data(dir, limit_train_images, non_chars, train_data)
get_data(val_dir, limit_val_images, non_chars, val_data)

print(len(train_data))
print(len(val_data))

In [ ]:
random.shuffle(train_data)
random.shuffle(val_data)

In [ ]:
train_X = []
train_Y = []
for features,label in train_data:
    train_X.append(features)
    train_Y.append(label)

In [ ]:
val_X = []
val_Y = []
for features,label in val_data:
    val_X.append(features)
    val_Y.append(label)

In [ ]:
# normalizing the data
train_X = np.array(train_X)/255.0
train_X = train_X.reshape(-1,32,32,1)
train_Y = np.array(train_Y)

val_X = np.array(val_X)/255.0
val_X = val_X.reshape(-1,32,32,1)
val_Y = np.array(val_Y)

print(train_X.shape,val_X.shape)
print(train_Y.shape,val_Y.shape)

In [ ]:
# Label binarization
LB = LabelBinarizer()
train_Y = LB.fit_transform(train_Y)
val_Y = LB.fit_transform(val_Y)
LB.classes_

#save as the label file
np.save('label_classes.npy', LB.classes_)

For test:

In [ ]:
# loading random 1000 rows (texts) from csv for testing
handwritten_test_data = pd.read_csv("datasets/handwriting-recognition/written_name_test_v2.csv")
handwritten_test_data = handwritten_test_data.sample(n=1000)

# loading 1000 images from test dataset for loaded texts
handwritten_dir = "datasets/handwriting-recognition/test_v2/test/"
handwritten_test_data["image"] = ""

for index, row in handwritten_test_data.iterrows():
    img = cv2.imread(os.path.join(handwritten_dir,row["FILENAME"]),0)
    handwritten_test_data.at[index,"image"] = img

# preparing data
handwritten_test_X = []
handwritten_text_Y = []
for index, row in handwritten_test_data.iterrows():
    handwritten_test_X.append(row["FILENAME"])
    handwritten_text_Y.append(row["IDENTITY"])

**Experiments with models and layers**
- MaxPooling2D to use over AveragePooling2D as it is more robust to noise.
- Dropout to reduce overfitting.
- BatchNormalization to normalize the activations of the previous layers.

In [ ]:
    # tf.keras.layers.Reshape((128, 1)), #these 3 layers are for the RNN and increase time taken to train by a lit 2min -> 25min per epoch on CPU
    # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True, dropout=0.25)), #
    # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True, dropout=0.25)), #
    # tf.keras.layers.Flatten(),
    #tf.keras.layers.BatchNormalization(),
#maybe consider something like this for the RNN part of the model

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), strides=(1, 1), activation='gelu', padding='same', kernel_initializer='he_normal', input_shape=(img_size, img_size,1)))
model.add(MaxPooling2D())
model.add(SpatialDropout2D(0.4))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), strides=(1, 1), activation='gelu', padding='same', kernel_initializer='he_normal'))
model.add(MaxPooling2D())
model.add(SpatialDropout2D(0.4))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), strides=(1, 1), activation='gelu', padding='same', kernel_initializer='he_normal'))
model.add(SpatialDropout2D(0.4))
model.add(GlobalMaxPooling2D())
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(128, activation='gelu', kernel_initializer='he_normal'))
model.add(Dropout(0.4))
model.add(Dense(64, activation='gelu', kernel_initializer='he_normal'))
model.add(Dropout(0.4))
model.add(BatchNormalization())
model.add(Dense(25, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

In [ ]:
es = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 20, restore_best_weights=True)
history = model.fit(train_X,train_Y, epochs=60, batch_size=32, validation_data = (val_X, val_Y),  verbose=1, callbacks=[es])

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Training Accuracy vs Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Training Loss vs Validation Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
#save model
model.save("textReadModel(handwritten).keras")

Testing the model on recognition set

In [ ]:
def sort_contours(cnts, method="left-to-right"):
    reverse = False
    i = 0
    if method == "right-to-left" or method == "bottom-to-top":
        reverse = True
    if method == "top-to-bottom" or method == "bottom-to-top":
        i = 1
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
    key=lambda b:b[1][i], reverse=reverse))
    # return the list of sorted contours and bounding boxes
    return (cnts, boundingBoxes)

In [ ]:
def get_letters(img):
    letters = []
    image = cv2.imread(img)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret,thresh1 = cv2.threshold(gray ,127,255,cv2.THRESH_BINARY_INV)
    dilated = cv2.dilate(thresh1, None, iterations=2)

    cnts = cv2.findContours(dilated.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = sort_contours(cnts, method="left-to-right")[0]
    # loop over the contours
    for c in cnts:
        if cv2.contourArea(c) > 10:
            (x, y, w, h) = cv2.boundingRect(c)
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        roi = gray[y:y + h, x:x + w]
        thresh = cv2.threshold(roi, 0, 255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
        thresh = cv2.resize(thresh, (32, 32), interpolation = cv2.INTER_CUBIC)
        thresh = thresh.astype("float32") / 255.0
        thresh = np.expand_dims(thresh, axis=-1)
        thresh = thresh.reshape(1,32,32,1)
        ypred = model.predict(thresh)
        ypred = LB.inverse_transform(ypred)
        [x] = ypred
        letters.append(x)
    return letters, image

#plt.imshow(image)

In [ ]:
def get_word(letter):
    word = "".join(letter)
    return word

In [ ]:
letter,image = get_letters("datasets/handwriting-recognition/train_v2/train/TRAIN_00023.jpg")
word = get_word(letter)
print(word)
plt.imshow(image)

In [ ]:
letter,image = get_letters("datasets/handwriting-recognition/train_v2/train/TRAIN_00003.jpg")
word = get_word(letter)
print(word)
plt.imshow(image)

In [ ]:
letter,image = get_letters("datasets/handwriting-recognition/train_v2/train/TRAIN_00030.jpg")
word = get_word(letter)
print(word)
plt.imshow(image)

In [ ]:
letter,image = get_letters("datasets/handwriting-recognition/test_v2/test/TEST_0007.jpg")
word = get_word(letter)
print(word)
plt.imshow(image)

In [ ]:
# loading random 1000 rows from csv 
test_data = pd.read_csv("datasets/handwriting-recognition/written_name_test_v2.csv")
test_data = test_data.sample(n=1000)

In [ ]:
# prediction method for testing
def predict_test(filename, label):
    dir = "datasets/handwriting-recognition/test_v2/test/"
    letter,image = get_letters(os.path.join(dir,filename))
    word = get_word(letter)
    print("predicted: " + word)
    print("actual: " + label)
    plt.imshow(image)

In [ ]:
# prediction test
index = random.randint(0,999)
filename = handwritten_test_X[index]
label = handwritten_text_Y[index]
predict_test(filename, label)